<a href="https://colab.research.google.com/github/RafaelGallo/GAN_Generative-Adversarial-Network_DeOldify/blob/main/GAN_Generative_Adversarial_Network_DeOldify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **GAN Generative Adversarial Network DeOldify**

In [ ]:
# Clone the DeOldify repository to the 'DeOldify' folder
!git clone https://github.com/jantic/DeOldify.git DeOldify

In [ ]:
# Change to the DeOldify directory (only necessary in a terminal, not in a notebook)
!cd DeOldify

In [ ]:
# Set GPU0 as the processing device (if available)
device.set(device=DeviceId.GPU0)

# Check if GPU is available, print a warning if not
if not torch.cuda.is_available():
    print('GPU not available.')

In [ ]:
# Install all requirements for DeOldify on Colab
!pip install -r requirements-colab.txt

In [ ]:
# Optimize PyTorch for fast GPU usage (cudnn benchmark)
torch.backends.cudnn.benchmark = True

In [ ]:
# Create the 'models' directory to store weights
!mkdir 'models'

In [ ]:
# Download the Stable DeOldify model to the 'models' folder
!wget https://www.dropbox.com/s/axsd2g85uyixaho/ColorizeStable_gen.pth?dl=0 -O ./models/ColorizeStable_gen.pth

In [ ]:
# Adjust references for DeOldify internal classes
collections.Sized = collections.abc.Sized
collections.Mapping = collections.abc.Mapping
collections.Iterable = collections.abc.Iterable

# Add safe types for PyTorch model deserialization
torch.serialization.add_safe_globals([functools.partial, builtins.getattr, slice])

In [ ]:
# (Re) create the models folder (-p avoids error if already exists)
!mkdir -p models/

In [ ]:
# Download the Artistic DeOldify model to the 'models' folder
!wget https://data.deepai.org/deoldify/ColorizeArtistic_gen.pth -O ./models/ColorizeArtistic_gen.pth

In [ ]:
# PyTorch and fastai
import torch
import fastai

# DeOldify imports
from deoldify import device
from deoldify.device_id import DeviceId
from deoldify.visualize import *

# Python standard libraries
import os
import shutil
from pathlib import Path
from PIL import Image
import functools
import builtins
import collections
import collections.abc

In [ ]:
# Create the DeOldify artistic colorizer object
colorizer = get_image_colorizer(artistic=True)

In [ ]:
# Replace with the path to your grayscale image
uploaded_filename = '/content/16034886295f934b754954d_1603488629_3x4_md.jpg'

In [ ]:
# Set the render_factor (higher = more color, but can produce artifacts)
render_factor = 15

# Generate the colorized image and display the comparison (saved to DeOldify's default folder)
result_path = colorizer.plot_transformed_image(
    Path(uploaded_filename),
    render_factor=render_factor,
    compare=True
)

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os
from pathlib import Path
import glob

# List of image paths you want to colorize
input_imgs = ['/content/tarsila-do-amaral-1554419161395_v2_800x1125.jpg',
              '/content/SITE_Enchentes-0-2280px-abre.jpg',
              '/content/16034886295f934b754954d_1603488629_3x4_md.jpg',
              '/content/97167585_SCSemana-de-22Oswald-de-Andrade-e-Tarsila-do-Amaral-a-bordo-de-um-navio.Credito-Fun.jpg',
              '/content/61-praca-da-republica-em-1915-1453570391749_956x500.jpg.webp',
              '/content/52-avenida-paulista-em-1930-1453570448145_956x500.jpg.webp',
              '/content/95-edificio-altino-arantes-conhecido-como-predio-do-banespa-ou-banespao-e-inspirado-no-empire-state-de-nova-york-foi-inaugurado-em-1947-na.webp',
              '/content/mario.jpeg',
              '/content/001.png',
              '/content/estacaodaluz.jpg',
              '/content/50-avenida-paulista-em-1902-1453570453709_956x500.jpg.webp',
              '/content/dom-pedro-i.jpg',
              '/content/dom_pedro22.jpeg',
              '/content/dpedro2.jpg',
              '/content/familiareal2.jpg',
              '/content/sp45.jpg']

In [ ]:
# Pasta de saída dos resultados coloridos e dos comparativos
output_dir = '/content/teste_render_factors'
compare_dir = os.path.join(output_dir, 'comparisons')
os.makedirs(compare_dir, exist_ok=True)

In [ ]:
for img_path in input_imgs:
    basename = Path(img_path).stem
    # Busca todos os arquivos coloridos do tipo *_rf*.jpg (NUNCA *_compare.jpg)
    pattern = os.path.join(output_dir, f"{basename}_rf*.jpg")
    colorized_files = [f for f in glob.glob(pattern) if '_compare' not in f and '_before_after' not in f]
    if not colorized_files:
        print(f"No colorized images found for: {basename}")
        continue

    # Abre o original SEMPRE em preto-e-branco
    img_original = Image.open(img_path).convert('L').convert('RGB')

    for colorized_path in colorized_files:
        rf_part = Path(colorized_path).stem.split('_rf')[-1]
        compare_path = os.path.join(compare_dir, f"{basename}_rf{rf_part}_before_after.jpg")

        img_colorized = Image.open(colorized_path).convert('RGB')
        img_colorized = img_colorized.resize(img_original.size)

        # Plota e salva lado a lado: Original (B&W) | Colorized
        plt.figure(figsize=(10, 7))
        plt.subplot(1, 2, 1)
        plt.imshow(img_original)
        plt.axis('off')
        plt.title("Original", fontsize=14)

        plt.subplot(1, 2, 2)
        plt.imshow(img_colorized)
        plt.axis('off')
        plt.title(f"Colorized (rf={rf_part})", fontsize=14)

        plt.suptitle("Colorized", fontsize=20)
        plt.tight_layout()
        plt.savefig(compare_path, bbox_inches='tight')
        plt.show()
        plt.close()
        print(f"Saved: {compare_path}")